In [ ]:
import os
import numpy as np
import suite2p
from tempfile import TemporaryDirectory
from pathlib import Path

### set parameters

In [ ]:
ops = suite2p.default_ops()
print(ops)

### set data path

In [ ]:
db = {
    'data_path': [os.path.join('/Users', 'shogoakiyama', 'Desktop')],
    'save_path0': TemporaryDirectory().name,
    'tiff_list': ['Sue_2x_3000_40_-46.tif'],
}
db

### run suite2p

In [ ]:
output_ops = suite2p.run_s2p(ops=ops, db=db)

### result

In [ ]:
output_op = output_ops
print('param:\n', set(output_op.keys()).difference(ops.keys()))
print('\noutput path: \n', list(Path(output_op['save_path']).iterdir()))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 4))
plt.subplot(1, 4, 1)
plt.imshow(output_op['refImg'], cmap='gray', )
plt.title("Reference Image for Registration");

plt.subplot(1, 4, 2)
plt.imshow(output_op['max_proj'], cmap='gray')
plt.title("Registered Image, Max Projection");

plt.subplot(1, 4, 3)
plt.imshow(output_op['meanImg'], cmap='gray')
plt.title("Mean registered image")

plt.subplot(1, 4, 4)
plt.imshow(output_op['meanImgE'], cmap='gray')
plt.title("High-pass filtered Mean registered image");

In [ ]:
plt.imshow(output_op['refImg'])

In [ ]:
plt.imshow(output_op['meanImg'])

### roi detection

In [ ]:
from suite2p import ROI

stats_file = Path(output_op['save_path']).joinpath('stat.npy')
iscell = np.load(Path(output_op['save_path']).joinpath('iscell.npy'), allow_pickle=True)[:, 0].astype(bool)
stats = np.load(stats_file, allow_pickle=True)
stats.shape, iscell.shape

In [ ]:
label_id = True
arrays = []
for i, s in enumerate(stats):
    array = ROI(ypix=s['ypix'], xpix=s['xpix'], lam=s['lam'], med=s['med'], do_crop=False).to_array(Ly=output_op['Ly'], Lx=output_op['Lx'])
    if label_id:
        array *= i + 1
    arrays.append(array)
im = np.stack(arrays)
import copy
nan_im = copy.deepcopy(im)
nan_im[im == 0] = np.nan

In [ ]:
plt.figure(figsize=(16, 4))

plt.subplot(1, 4, 1)
plt.imshow(output_op['meanImg'], cmap='gray')
plt.title("Registered Image, Max Projection")

plt.subplot(1, 4, 2)
plt.imshow(np.nanmax(nan_im, axis=0), cmap='jet')
plt.title("All ROIs Found")

plt.subplot(1, 4, 3)
plt.imshow(np.nanmax(nan_im[~iscell], axis=0, ), cmap='jet')
plt.title("All Non-Cell ROIs")

plt.subplot(1, 4, 4)
plt.imshow(np.nanmax(nan_im[iscell], axis=0), cmap='jet')
plt.title("All Cell ROIs");

In [ ]:
print(output_op['max_proj'].shape)
print(output_op['meanImg'].shape)
print(nan_im.shape)

In [ ]:
tmp_im = im.reshape(-1, 170*170).transpose(1, 0)
tmp_im.shape

from scipy.sparse import csc_matrix
sparse_roi = csc_matrix(tmp_im)
sparse_roi

In [ ]:
from caiman.utils.visualization import get_contours
coordinates = get_contours(sparse_roi, (170, 170))

In [ ]:
# pl.imshow(Cn)
sparse_rois = []
for i, c in enumerate(coordinates):
    if iscell[i]:
        plt.plot(*c['coordinates'].T)
    sparse_rois.append(c['coordinates'].T)
# plt.gca().invert_xaxis()
# plt.gca().invert_yaxis()

In [ ]:
# plt.figure(figsize=(16, 4))

plt.imshow(output_op['meanImg'], cmap='gray')
plt.title("Registered Image, Max Projection")

plt.imshow(np.nanmax(nan_im[iscell], axis=0), cmap='jet')
plt.title("All Cell ROIs");

In [ ]:
inv_im = sparse_roi.toarray().reshape(170, 170, -1).transpose(2, 0, 1)
inv_im *= 255
inv_im[inv_im == 0] = np.nan

In [ ]:
plt.imshow(np.nanmax(inv_im[iscell], axis=0), cmap='jet')

In [ ]:
print(inv_im[~np.isnan(inv_im)].shape)
print(inv_im[np.isnan(inv_im)].shape)

In [ ]:
print(output_op['max_proj'].shape)
print(np.nanmax(nan_im[iscell], axis=0).shape)

### Extract trace

In [ ]:
f_cells = np.load(Path(output_op['save_path']).joinpath('F.npy'))
f_neuropils = np.load(Path(output_op['save_path']).joinpath('Fneu.npy'))
spks = np.load(Path(output_op['save_path']).joinpath('spks.npy'))
f_cells.shape, f_neuropils.shape, spks.shape

In [ ]:
plt.figure(figsize=[20,60])
plt.suptitle("Flourescence and Deconvolved Traces for Different ROIs", y=0.92);
rois = np.arange(len(f_cells))[::20]
for i, roi in enumerate(rois):
    plt.subplot(len(rois), 1, i+1, )
    f = f_cells[roi]
    f_neu = f_neuropils[roi]
    sp = spks[roi]
    # Adjust spks range to match range of fluroescence traces
    fmax = np.maximum(f.max(), f_neu.max())
    fmin = np.minimum(f.min(), f_neu.min())
    frange = fmax - fmin 
    sp /= sp.max()
    sp *= frange
    plt.plot(f, label="Cell Fluorescence")
    plt.plot(f_neu, label="Neuropil Fluorescence")
    plt.plot(sp + fmin, label="Deconvolved")
    plt.xticks(np.arange(0, f_cells.shape[1], f_cells.shape[1]/10))
    plt.ylabel(f"ROI {roi}", rotation=0)
    plt.xlabel("frame")
    if i == 0:
        plt.legend(bbox_to_anchor=(0.93, 2))